In [1]:
import json
import psycopg2
from psycopg2.extras import execute_batch
from pathlib import Path
from sentence_transformers import SentenceTransformer
from psycopg2.extras import execute_values

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Database connection parameters
DB_PARAMS = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": "5433"  # Default is 5432
}

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**DB_PARAMS)
    conn.autocommit = True
    # cursor = conn.cursor()
except Exception as e:
    print(f"Error connecting to database: {e}")
    exit(1)

init_pgvector = "CREATE EXTENSION IF NOT EXISTS vector;" # execute this first
create_table = """CREATE TABLE "example" (id bigserial PRIMARY KEY, keyword VARCHAR(100), embedding vector(384));"""
insert_embeddings = "INSERT INTO example (keyword, embedding) VALUES %s"

c:\Users\User\alejo\vector_db\vector_db\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
#limite maximo es de 384 tokens
sentences = ["hello world", "hello python", "bye python", "la girafa es roja", "python", "bython", "el mico esta en nz", "mico god", "el mico es cafe", "yom es un lloron", "a yom nada le gusta"]
embeddings = model.encode(sentences, normalize_embeddings=True).tolist()

In [2]:
cur = conn.cursor()
#install pgvector
cur.execute(init_pgvector)
conn.commit()


In [7]:
cursor = conn.cursor()
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cursor.fetchall())

[('example',)]


In [23]:
len(sentences)

11

In [8]:
# embeddings = model.encode(sentences, normalize_embeddings=True).tolist()
data = [(sentence, embedding) for sentence, embedding in zip(sentences, embeddings)]
with conn.cursor() as curs:
    execute_values(
                curs,
                insert_embeddings,
                data,
                template="(%s, %s)"
            )

In [10]:
data = str(model.encode("python", normalize_embeddings=False).tolist())
with conn.cursor() as curs:
    curs.execute("""
            SELECT keyword,
            1-(embedding <=> %(data)s) as similarity
            FROM example
            --WHERE 1-(embedding <=> %(data)s) >= 0.5
            ORDER BY embedding <=> %(data)s
            LIMIT 5;
        """,
        {"data": data}
    )
    result = [(keyword) for keyword in curs.fetchall()]
result

[('python', 1.0),
 ('python', 1.0),
 ('hello python', 0.8450508713722229),
 ('hello python', 0.8450508713722229),
 ('bye python', 0.6831845045089722)]

In [88]:
with conn.cursor() as curs:
    curs.execute("CREATE INDEX ON example USING ivfflat (embedding vector_cosine_ops) WITH (lists = 100);")

In [92]:
# download pre-chunked text and pre-computed embeddings
# this file is ~200 MB, so may take a minute depending on your connection speed
import pandas as pd
embeddings_path = "https://cdn.openai.com/API/examples/data/winter_olympics_2022.csv"

df = pd.read_csv(embeddings_path)

In [98]:
df.loc[1000,"text"]

"Stefania Constantini\n\n==Career==\n\nConstantini made her international curling debut at the [[Curling at the 2016 Winter Youth Olympics – Mixed team|2016 Winter Youth Olympics]] in the mixed team as third for [[Luca Rizzolli]]. The team went 3–4 through the round robin but lost in a tiebreaker to Turkey's [[Oğuzhan Karakurt]]. She also competed in the [[Curling at the 2016 Winter Youth Olympics – Mixed doubles|mixed doubles event]] with British curler [[Callum Kinnear]]. The pair made it to the final eight, where they lost to China's [[Zhao Ruiyi]] and Norway's [[Andreas Hårstad]].\n\nConstantini was added to the Italian National Women's Curling Team for the [[2017–18 curling season|2017–18 season]] as the team's second. The team also consisted of skip [[Diana Gaspari]], third [[Veronica Zappone]], lead [[Angela Romei]], and alternate [[Chiara Olivieri]]. The team's first international event of the season came at the [[2017 European Curling Championships]] in [[St. Gallen]], [[Switz